In [1]:
import json
import pandas as pd

word_list = []
with open(r"word_list\spanish\es_wordlist.json") as file:
    json_data = json.load(file)
    for d in json_data.items():
        word_list.append([d[0], d[1]])

In [23]:
df = pd.DataFrame(word_list)
df.columns = ["word", "count"]

# only lowercase word
df["word"] = df["word"].astype(str).apply(lambda x: x.lower())
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

# allowed alphabets
df = df[~df["word"].str.contains("[ãçëîôõöü]")]
df = df[df["word"].str.isalpha()]

# frequent words
df = df[df["count"] > df["count"].quantile(0.90)]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,count,word_len
0,a,22256821,1
1,aa,2017,2
2,aaa,1331,3
3,aaaron,31,6
4,aaarp,29,5


In [24]:
df.dtypes

word        object
count        int64
word_len     int64
dtype: object

In [25]:
df.describe()

,count,word_len
count,2.119180e+05,211918.000000
mean,4.689581e+03,8.198138
std,2.482541e+05,3.056647
min,2.700000e+01,1.000000
25%,4.700000e+01,6.000000
50%,1.060000e+02,8.000000
75%,4.220000e+02,10.000000
max,7.534259e+07,62.000000


In [31]:
df.query("count > 10 and word_len == 4").describe()

,count,word_len
count,1.115900e+04,11159.0
mean,6.332420e+03,4.0
std,1.015553e+05,0.0
min,2.700000e+01,4.0
25%,4.800000e+01,4.0
50%,1.090000e+02,4.0
75%,4.105000e+02,4.0
max,7.993370e+06,4.0


In [32]:
df.query("'libros' in word")

,word,count,word_len
120229,libros,27544,6


In [35]:
difficulties = {"easy": 0.965, "medium": 0.9, "hard": 0.7, "all": 0}
lengths = [4]

for l in lengths:
    for name, percentile in difficulties.items():
        cutoff = df["count"].quantile(percentile)
        df_filtered = (
            df.query(f"count > {cutoff} and word_len == {l}")
            .sort_values(by=["count"], ascending=False)
            .reset_index(drop=True)
        )
        with open(rf"vocabs\es\es-len{l}-{name}.json", "w") as f:
            data = df_filtered["word"].sort_values().tolist()
            json.dump(data, f)
            print(f"{f.name}: {len(df_filtered)} words")

vocabs\es\es-len4-easy.json: 438 words
vocabs\es\es-len4-medium.json: 1148 words
vocabs\es\es-len4-hard.json: 3312 words
vocabs\es\es-len4-all.json: 10978 words
